## Segmenting and Clustering Neighborhoods in Toronto

## Import libraries

In [1]:
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

from bs4 import BeautifulSoup 
import urllib3

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

# Canadian Postal Code Database
import geocoder

print('Libraries imported.')

Libraries imported.


## Pull Data from URL

In [2]:
http = urllib3.PoolManager()
r = http.request('GET', 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
bs = BeautifulSoup(r.data, "html.parser")

/Users/kathleenprince-sayward/anaconda/lib/python3.6/site-packages/urllib3/connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [3]:
data = []
table = bs.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values

### Convert Data to dataframe & Clean up

In [4]:
df = pd.DataFrame(data, columns=['Postcode','Borough','Neighborhood'])
df = df[df.Borough != 'Not assigned']
df.Neighborhood.replace('Not assigned',df.Borough,inplace=True)
df = df.groupby('Postcode').agg(lambda x: ','.join(set(x)))
df = df.reset_index()
df.shape

(103, 3)

In [5]:
df.head()

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union"
2,M1E,Scarborough,"Morningside,West Hill,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


### Can't get the geocoder to work :(

In [6]:
def get_geocoder(postal_code_from_df):
     # initialize your variable to None
    lat_lng_coords = None
     # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code_from_df))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

#### Using the CSV instead

In [7]:
df_loc = pd.read_csv('./data/Geospatial_Coordinates.csv')

In [8]:
df_loc.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
df2 = df.merge(df_loc, left_on='Postcode', right_on='Postal Code', how='left')
df2.head()

,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Morningside,West Hill,Guildwood",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [10]:
df2 = df2.drop(['Postal Code'], axis=1)
df2.head()

,Postcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill,Highland Creek,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
